# Example 0

This notebook generates dark neutrino events, loads them, and describes the output formats. 

In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import os
import numpy as np
import pandas as pd

import DarkNews as dn
from DarkNews import const

## Generating events

Let us start by generating a few dark neutrino events for two cases of interest.

We do this in two different ways. 

First using the GenLauncher class, as well as via the command line functionality. In the second case we will need to import the dataframe or the HEPEVT file from the specified path.

In [20]:
cmd_string = "dn_gen --mzprime=1.25 --m4=0.140 --neval=1000 --D_or_M=dirac --log=INFO"
const.subprocess_cmd(cmd_string)

cmd_string = "dn_gen Umu5=1e-3 UD5=35 --chi=0.0031 --gD=2 --mzprime=1.25 --m4=0.080 --m5=0.140 --neval=1000 --D_or_M=majorana --log=INFO"
const.subprocess_cmd(cmd_string)

### DANIELE's METHOD....
# Gen = ....
# df_m = Gen.run(...)

dn_gen --mzprime=1.25 --m4=0.140 --neval=1000 --D_or_M=dirac --log=INFO
9.866349020787781e-14
0.00299792458


    #########################################################
    #   ______           _        _   _                     #
    #   |  _  \         | |      | \ | |                    #
    #   | | | |__ _ _ __| | __   |  \| | _____      _____   #
    #   | | | / _  | ___| |/ /   | .   |/ _ \ \ /\ / / __|  #
    #   | |/ / (_| | |  |   <    | |\  |  __/\ V  V /\__ \  #
    #   |___/ \__,_|_|  |_|\_\   \_| \_/\___| \_/\_/ |___/  #
    #                                                       #
    #########################################################
    
Theory model used: 3+1 dirac HNL model


Generating helicity conserving upscattering events for:
	nu(mu) + C12 -> N4 +  C12 -> nu(mu) + e+ + e- + C12

decaying N4 using off-shell mediator.
Generating helicity flipping upscattering events for:
	nu(mu) + C12 -> N4 +  C12 -> nu(mu) + e+ + e- + C12

decaying N4 using off-shell me

In [12]:
# loading the events generated via the command line
df_d = pd.read_pickle('data/miniboone/3plus1/m4_0.14_mzprime_1.25_dirac/pandas_df.pckl')

FileNotFoundError: [Errno 2] No such file or directory: 'data/miniboone/3plus2/m5_0.14_m4_0.08_mzprime_1.25_dirac/pandas_df.pckl'

---
# Pandas DataFrame Output

### Metadata

Before inspecting the events inside the pandas dataframe, let us check the metadata in [df.attrs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.attrs.html). We have:

---
1. **experiment**: this is a class DarkNews.experiment.Detector, which contains all the information about the experiment for which the events are generated. It also contains a list of instances of the NuclearTarget class, which contains information about the different scattering target used.

In [242]:
# general description of the detector attribute
keys = list(df_d.attrs['experiment'].__dict__.keys())
print(f"Attributes of Detector class: \n",*keys)
print(f"\nAttributes of NuclearTarget class: \n",*list(df_d.attrs['experiment'].NUCLEAR_TARGETS[0].__dict__))

# how much of each target inside the detector?
print("\nExample of how to look up nuclear targets:")
for target, fid_mass in zip(df_d.attrs['experiment'].NUCLEAR_TARGETS, df_d.attrs['experiment'].FIDUCIAL_MASS_PER_TARGET):
    print(fr"{round(fid_mass,2)} tonnes of {target.name}")


Attributes of Detector class: 
 NAME FLUXFILE FLUX_NORM ERANGE NUCLEAR_TARGETS FIDUCIAL_MASS_PER_TARGET NUMBER_OF_TARGETS POTS

Attributes of NuclearTarget class: 
 name Z N A atomic_Eb electronic_Eb nuclear_Eb atomic_mass excess_mass nuclear_mass beta_decay_energy mass charge is_hadron is_nucleus is_proton is_neutron is_nucleon is_free_nucleon is_bound_nucleon pdgid tau3 F1_EM F2_EM F1_NC F2_NC F3_NC

Example of how to look up nuclear targets:
701.14 tonnes of C12
116.86 tonnes of H1


---
2. **bsm_model**: this attribute contains DarkNews.model.Model, which stores all the new physics parameters used in the generation. This includes both low-level information, such as the couplings, say $g_D$, as well as high-level ones, such as the couplings of the $Z^\prime$ to electrons, $d_e^V$ and $d_e^A$.

In [244]:
print("Some low-level parameters:\n",*list(df_d.attrs['bsm_model'].__dict__.keys())[:19])
print("\nSome high-level parameters:\n",*list(df_d.attrs['bsm_model'].__dict__.keys())[-14:])

Some low-level parameters:
 gD epsilon epsilonZ Ue4 Umu4 Utau4 Ue5 Umu5 Utau5 Ue6 Umu6 Utau6 UD4 UD5 UD6 m4 m5 m6 mzprime

Some high-level parameters:
 deV deA duV duA ddV ddA dVproton dAproton dVneutron dAneutron cVproton cAproton cVneutron cAneutron


---
### Description of the event structure

Now we look at the actual events. The DataFrame is a MultiIndex df. Each event contains the components for all the 4-momenta of the particles involved:

$$\nu _\text{P_projectile} \,+\, \text{Hadronic target} _\text{P_target} \to   N _\text{P_decay_N_parent} \,+\, \text{Hadronic recoil} _\text{P_recoil}$$

$$N _\text{P_decay_N_parent} \to N^\prime _\text{P_decay_N_daughter} \,+\, \ell^+ _\text{P_decay_ell_plus} \,+\, \ell^-_\text{P_decay_ell_minus}$$



In [28]:
df_d.dtypes

P_projectile        t    float64
                    x    float64
                    y    float64
                    z    float64
P_decay_N_parent    t    float64
                    x    float64
                    y    float64
                    z    float64
P_target            t    float64
                    x    float64
                    y    float64
                    z    float64
P_recoil            t    float64
                    x    float64
                    y    float64
                    z    float64
P_decay_ell_minus   t    float64
                    x    float64
                    y    float64
                    z    float64
P_decay_ell_plus    t    float64
                    x    float64
                    y    float64
                    z    float64
P_decay_N_daughter  t    float64
                    x    float64
                    y    float64
                    z    float64
decay_displacement  t    float64
                    x    float64
          

##

* w_decay_rate_0           float64 -- weight of the decay rate (sum(w) = Gamma_N)
* I_decay_rate_0           float64 -- total rate Gamma_N


* w_event_rate             float64 -- weight for the event rate (sum(w) = event rate)
* I_event_rate             float64 -- total event rate


* w_flux_avg_xsec          float64 -- weight of the flux averaged cross section (sum(w) = int(sigma*flux)* exposure)
* I_flux_avg_xsec          float64 -- int(sigma*flux)* exposure


* target                    object -- target object -- it will typically be a nucleus
* target_pdgid               int64 -- pdgID of the target 


* scattering_regime         object -- regime can be coherent or p-elastic
* helicity                  object -- helicity process, can be flipping or conserving. flipping is suppressed
* underlying_process        object -- string of the underlying process, e.g, "nu(mu) + proton_in_C12 -> N4 +  proton_in_C12 -> nu(mu) + e+ + e- + proton_in_C12"